In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Mar 30 08:35:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install tokenizers

     |████████████████████████████████| 3.2MB 12.8MB/s 


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.0MB 15.8MB/s 
     |████████████████████████████████| 890kB 51.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=af296d01f3164b58fcb8be25ae2afd759df7125975921a4d641b6b919d28c5a2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 1.5MB 13.5MB/s 
Requirement already up-to-date: setuptools in /usr/local/lib/python3.7/dist-packages (54.2.0)
Requirement already up-to-date: wheel in /usr/local/lib/python3.7/dist-packages (0.36.2)
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 12.8 MB 15.3 MB/s 
     |████████████████████████████████| 1.1 MB 80.3 MB/s 
     |████████████████████████████████| 456 kB 60.7 MB/s 
     |████████████████████████████████| 9.1 MB 54.6 MB/s 
     |████████████████████████████████| 113 kB 60.1 MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107097 sha256=19b186ba29ad9aa8b64e578eea7ba0045629468e951df1ecbf5cdc724ab01123
  Stored in directory: /root/.cache/pip/wheels/83/a6/12/bf3c1a667bde4251be5b7a3368b2d604c9af2105b5c1cb1870
Successfully built smart-open
  Attempting uninstall: catalogue
    Found existing installa

In [ ]:
import pandas as pd
import os
import pickle as pkl
import numpy as np
import gzip
from tqdm import tqdm_notebook as tqdm
import spacy
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
# configuration = BertConfig(num_hidden_layers=2,num_attention_heads=4)
# encoder = TFBertModel.from_pretrained("bert-base-uncased")

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
df=pd.read_csv("/content/dev.csv")

In [ ]:
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]

In [ ]:
# ### LOAD BERT TOKENIZERS ###
# # Save the slow pretrained tokenizer
# slow_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# save_path = "bert_base_uncased/"
# if not os.path.exists(save_path):
#     os.makedirs(save_path)
# slow_tokenizer.save_pretrained(save_path)

# # Load the fast tokenizer from saved file
# tokenizer = BertWordPieceTokenizer("bert_base_uncased/vocab.txt", lowercase=True)

In [ ]:
from collections import Counter
import functools
import operator

In [ ]:
spacy.prefer_gpu()

In [ ]:
encoder = TFBertModel.from_pretrained("bert-base-cased")

In [ ]:
x_ids_tr=[tokenizer.encode(i).ids for i in train["wrong"]]
x_ids_tr=pad_sequences(x_ids_tr,465,padding="post")
x_msk_tr=[tokenizer.encode(i).attention_mask for i in train["wrong"]]
x_msk_tr=pad_sequences(x_msk_tr,465,padding="post")
y_tr=[tokenizer.encode(i).ids for i in train["correct"]]
y_tr=pad_sequences(y_tr,465,padding="post")


In [ ]:
x_ids_va=[tokenizer.encode(i).ids for i in test["wrong"]]
x_ids_va=pad_sequences(x_ids_va,465,padding="post")
x_msk_va=[tokenizer.encode(i).attention_mask for i in test["wrong"]]
x_msk_va=pad_sequences(x_msk_va,465,padding="post")
y_va=[tokenizer.encode(i).ids for i in test["correct"]]
y_va=pad_sequences(y_va,465,padding="post")


NameError: ignored

In [ ]:
wrd_lst=functools.reduce(operator.iconcat, y_tr, [])+functools.reduce(operator.iconcat, y_va, [])

In [ ]:
wrd_lst=list(set(wrd_lst))

In [ ]:
wrd_dic={wrd:key+1 for key,wrd in enumerate(wrd_lst)}

In [ ]:
del wrd_lst

In [ ]:
y_tr=[list(map(lambda x:wrd_dic[x],i)) for i in y_tr]
y_va=[list(map(lambda x:wrd_dic[x],i)) for i in y_va]

KeyError: ignored

In [ ]:
input_ids = tf.keras.layers.Input(shape=(465,), dtype=tf.int32)
attn_mask = tf.keras.layers.Input(shape=(465,), dtype=tf.int32)
wrd_embedding = encoder(input_ids, attention_mask=attn_mask)[0]
den=tf.keras.layers.Dense(tokenizer.get_vocab_size()+1, activation='softmax')(wrd_embedding)
model=tf.keras.Model(inputs=[input_ids,attn_mask],outputs=den)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.00005),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=["accuracy"])

In [ ]:
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           [(None, 465)]        0                                            
__________________________________________________________________________________________________
input_32 (InputLayer)           [(None, 465)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 109482240   input_31[0][0]                   
                                                                 input_32[0][0]                   
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 465, 30523)   23472187    tf_bert_model[1][0]        

In [ ]:
callbacks=[
           tf.keras.callbacks.ModelCheckpoint("model.h5",monitor="val_accuracy",save_best_only=True,save_weights_only=True)
           ,tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=15)
]

In [ ]:
model.fit(x=[x_ids_tr,x_msk_tr],y=y_tr,epochs=5,verbose=1,batch_size=16,validation_data=([x_ids_va,x_msk_va],y_va),callbacks=callbacks)

Epoch 1/5
2853/2853 [==============================] - 443s 155ms/step - loss: 0.3156 - accuracy: 0.9538 - val_loss: 0.3154 - val_accuracy: 0.9537
Epoch 2/5
2853/2853 [==============================] - 445s 156ms/step - loss: 0.3130 - accuracy: 0.9538 - val_loss: 0.3146 - val_accuracy: 0.9537
Epoch 3/5
2853/2853 [==============================] - 447s 157ms/step - loss: 0.3114 - accuracy: 0.9539 - val_loss: 0.3139 - val_accuracy: 0.9539
Epoch 4/5
2056/2853 [====================>.........] - ETA: 1:44 - loss: 0.3094 - accuracy: 0.9545

KeyboardInterrupt: ignored

In [ ]:
model.save_weights('modelweight_contest2.h5')

In [ ]:
model.save("RNNmodel_contest2.h5")

In [ ]:
!gdown --id 16jWf59pcT1FR6Fl5NyW8NGOwrR-ron_w

Downloading...
From: https://drive.google.com/uc?id=16jWf59pcT1FR6Fl5NyW8NGOwrR-ron_w
To: /content/model.h5
523MB [00:03, 150MB/s]


Load Model to check

In [ ]:
model.load_weights("/content/model.h5")

NameError: ignored

In [ ]:
from tensorflow import keras
model = keras.models.load_model('/content/modelweight_contest2.h5')

In [ ]:
!gdown --id 1E62kjHzbwixY1In1AFlFgdCaD5ha90Td

Downloading...
From: https://drive.google.com/uc?id=1E62kjHzbwixY1In1AFlFgdCaD5ha90Td
To: /content/gec-test-set.txt
100% 162k/162k [00:00<00:00, 61.0MB/s]


In [ ]:
f1=open("gec-test-set.txt","r")
ts_set=[i for i in f1]

In [ ]:
print(ts_set)

In [ ]:
x_ids_ts=[tokenizer.encode(i).ids for i in ts_set]
x_ids_ts=pad_sequences(x_ids_ts,465,padding="post")
x_msk_ts=[tokenizer.encode(i).attention_mask for i in ts_set]
x_msk_ts=pad_sequences(x_msk_ts,465,padding="post")

In [ ]:
len(x_ids_ts)

1312

In [ ]:
len(x_msk_ts)

1312

In [ ]:
f1=open("gec-test-set.txt","r")
ts_set=[i for i in f1]
x_ids_ts=[tokenizer.encode(i).ids for i in ts_set]
x_ids_ts=pad_sequences(x_ids_ts,465,padding="post")
x_msk_ts=[tokenizer.encode(i).attention_mask for i in ts_set]
x_msk_ts=pad_sequences(x_msk_ts,465,padding="post")
x_ids_ts=np.array_split(x_ids_ts,200)
x_msk_ts=np.array_split(x_msk_ts,200)
x=list(zip(x_ids_ts,x_msk_ts))
list_eiei = []
for i in tqdm(x):
  pred_x=model.predict(i)
  for k,j in enumerate(pred_x[0]):
      print (j)
      lst=[]
      if (j.argmax()!=0):
        lst.append(tokenizer.decode([j.argmax()]))
      else:
        break
  list_eiei = list_eiei + lst

In [ ]:
print(list_eiei)

[]
